# Preprocessing: Water treatment plants

**Objective**: Prepare data from the **MERKUR** dataset for use in machine learning algorithms.

**Background**: The MERKUR project, based in the *Research Centre for Built Environment, Climate, Water Technology and Digitalisation* at VIA University College, collects and analyzes data from water treatment plants in Denmark. In short, the project aims to understand how water treatment plants are run, and the results are then ideally used to optimize the running of water treatment plants. However, the dataset is, as of now, relatively "dirty" in a machine learning context: There are many missing values, outliers, a mix of categorical and numeric data, etc.

**Data Source**: The dataset has kindly been provided to us by Senior Associate Professor Loren Mark Ramsay. You can read more [here](https://en.via.dk/research/built-environment-climate-water-technology-and-digitalisation/water-treatment-and-distribution) and [here](https://www.ucviden.dk/en/projects/merkur-national-web-baseret-dataplatform-til-drikkevandsbehandlin).

Note that we are only working with a subset of the full database. This subset is saved as an Excel file, `merkur.xlsx`.

#### Overall Instructions
1. Explore the dataset to understand the features and their distributions.
2. Preprocess the data, handling any missing values, outliers, etc.

Below some suggestions are given but the assignment is relatively "free".

Best of luck with your analysis!

### Suggestions

-  Filter out (i.e. remove) any irrelevant columns (e.g. names, IDs, etc.)
-  Several columns contain missing values (NaNs). Find out how large a percentage each column is missing. Perhaps some of them lack so much data that you should consider removing them?
-  Scale numeric data.
-  For the features you choose to keep, impute the missing values in an appropriate way - or perhaps you find it more appropriate to delete the rows?
-  Several features (e.g., "PrimaryTrigger") are categorical. Use one-hot encoding to turn them into numeric data. Be careful with the feature "Stages" - perhaps one-hot encoding is not the best choice here?
-  If you you choose to remove or replace outliers, do this now. If you choose to keep, move on.
-  Create a correlation matrix and discuss - based on this, you might want to drop certain columns.
-  Consider whether some features should be transformed (e.g. using log, square root etc.) and do this if found relevant.
-  There are only about 80 rows in the data set. Discuss consequences of this in terms of machine learning - as well as potential solutions. 
-  Think about whether there are other steps you find appropriate at this point. If not, declare your data set clean.

In [5]:
import pandas as pd

raw_data = pd.read_excel(r"merkur.xlsx")

raw_data

,WaterworksName,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,PrimaryTrigger,AverageTotalFilterDepth,OverallFilterGrainSizeMin,OverallFilterGrainSizeMax,...,SumOfld_layer,UniformityCoefficient,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,GravityPressureMixed,Stages,AbstractedVolume
0,Asnæs Vandværk,6,50.00,6.000000,5200.00000,10.00,Time,NaN,NaN,NaN,...,NaN,NaN,866.666667,0.414525,NaN,27.294778,NaN,Gravity,Single,253543
1,Assens Vandværk,4,110.00,4.908739,300.00000,28.00,Volume,310.0,0.8,5.0,...,2891.666665,NaN,40.743665,23.000062,30.557749,2.576075,NaN,Pressure,Double,612094
2,Astrup Vandværk - Esbjerg,4,120.00,15.343900,3800.00000,83.30,Volume,190.0,2.0,35.0,...,759.000000,1.491,249.217919,2.192086,32.791831,21.825600,NaN,Gravity,Double,682318
3,Astrup Vandværk - Skjern,2,NaN,16.000000,1200.00000,26.00,Volume,NaN,NaN,NaN,...,NaN,NaN,75.000000,2.166907,NaN,12.554300,NaN,Gravity,Single,156998
4,Avernakø Vandværk,2,NaN,1.495000,250.00000,4.50,Volume,NaN,1.6,32.0,...,NaN,NaN,167.224093,1.818182,NaN,52.220566,NaN,Gravity,Single,6710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Værket ved Regnemark,16,1426.00,47.940002,NaN,110.00,Mix,142.0,0.8,150.0,...,90.809524,NaN,NaN,0.178441,NaN,9.176959,39.682540,Gravity,Double,11454775
75,Værket ved Søndersø,20,1407.29,10.178760,4500.00000,57.00,Volume,190.0,0.8,8.0,...,1708.095230,1.625,442.097088,1.236822,51.086775,6.040393,NaN,Pressure,Single,11021800
76,Østerbyværket,6,250.00,14.752500,2331.50137,51.57,Time,170.0,1.4,4.0,...,842.857154,1.850,158.041107,4.446077,30.367735,22.367525,45.833333,Gravity,Single,423362
77,Østre Vandværk - Herning,12,NaN,37.437401,4400.00000,134.00,Volume,NaN,NaN,NaN,...,NaN,NaN,93.489396,35.386015,32.053507,NaN,NaN,Gravity,Double,1521895


### Data preperation

By looking at the data, we have observed that some of the rows had a lot of missing values. To find the distribution of where these missing values are located, we have decided to find the features with the most missing.

In [10]:

# - Overviwe of missing values in the dataset
missing_values = raw_data.isnull().sum()

# - Percentage of missing values in the dataset
missing_percentage = (missing_values / len(raw_data)) * 100

# * Create a new dataframe with the missing values
missing_summary = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage, 'Data Type': raw_data.dtypes})
missing_summary.sort_values(by='Percentage', ascending=False)

,Missing Values,Percentage,Data Type
UniformityCoefficient,51,64.556962,float64
SumOfld_layer,39,49.367089,float64
TotalEBCT,39,49.367089,float64
AverageFilterBedVolume,34,43.037975,float64
OverallFilterGrainSizeMin,34,43.037975,float64
OverallFilterGrainSizeMax,34,43.037975,float64
AverageTotalFilterDepth,33,41.772152,float64
TankExploitation,30,37.974684,float64
HLR_BW,27,34.177215,float64
Stage1HLR,21,26.582278,float64


Based upon this, we can see that columns like `UniformityCoefficient` and `SumOfId_layer` have more than 40% missing values, which might be too much to impute meaningfully.

Columns:
- `UniformityCoeffficient`
- `SumOfId_layer`
- `TotalEBCT`
- `AverageFilterBedVolume`
- `OverallFilterGrainSizeMin`
- `OverallFilterGrainSizeMax`
- `AverageTotalFilterDepth`

For columns with a lower percentage of missing values, we could fill in the missing data using the mean or median for numerical colummns:
- `AverageTypicalRunVolume`
- `AverageFilterArea`
- `Footprint`
- `TankCapacity`
- `UFRV`
- `AverageBackwashVolume`
- `BW%`
- `MaxTypicalFlow`
- `FilterExploitation`
- `Stage1HLR`
- `HLR_BW`
- `TankExplotation`

In [22]:

data_cleaned = raw_data.copy()

# ! Extract the columns with more than 40% missing values
columns_to_drop = missing_summary[missing_summary['Percentage'] > 40]

data_cleaned = data_cleaned.drop(columns=columns_to_drop.index)

# ! Drop irrelevant columns like `WaterworksName`
data_cleaned = data_cleaned.drop(columns=['WaterworksName'])

# * Impute missing values for numerical columns using the median
numerical_columns = data_cleaned.select_dtypes(include=['float64','int64']).columns
data_cleaned[numerical_columns] = data_cleaned[numerical_columns].fillna(data_cleaned[numerical_columns].median())

# * Impute missing values for categorical columns using the mode
categorical_columns = data_cleaned.select_dtypes(include=['object']).columns

for column in categorical_columns:
    mode_value = data_cleaned[column].mode()[0]
    data_cleaned[column] = data_cleaned[column].fillna(mode_value)

# - Check for missing values after imputation
missing_values_after_imputation = data_cleaned.isnull().sum().sum()

# * Show the number of missing values after imputation
display(missing_values_after_imputation)

# * Show the first 5 rows of the cleaned dataset
data_cleaned.head()


0

,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,PrimaryTrigger,FilterExploitation,AerationType,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,GravityPressureMixed,Stages,AbstractedVolume
0,6,50.0,6.000000,5200.0,10.0,Time,57.886530,Cascade,1.880000,1.388889,143.341204,866.666667,0.414525,30.889453,27.294778,19.402985,Gravity,Single,253543
1,4,110.0,4.908739,300.0,28.0,Volume,63.521586,Air injection,1.620000,11.204508,16.714412,40.743665,23.000062,30.557749,2.576075,19.402985,Pressure,Double,612094
2,4,120.0,15.343900,3800.0,83.3,Volume,64.908486,Bottom aeration,2.213333,3.935020,44.693822,249.217919,2.192086,32.791831,21.825600,19.402985,Gravity,Double,682318
3,2,120.0,16.000000,1200.0,26.0,Volume,63.279585,Cascade,2.073333,3.742502,194.588021,75.000000,2.166907,30.889453,12.554300,19.402985,Gravity,Single,156998
4,2,120.0,1.495000,250.0,4.5,Volume,63.279585,Cascade,1.313333,3.742502,412.300024,167.224093,1.818182,30.889453,52.220566,19.402985,Gravity,Single,6710


### Impute missing values

We have decided to impute the values for the columns, which were missing below `40%` of values.

**Numerical values**
For the columns with numerical data, we opted to impute these missing enttries using the **median** of each respective column. The median, which is the middle value in a list of numbers, is less sensitive to outliers in the data compared to the mean. This would be the perferred choice for imputation in datasets, where outliers might skew the mean siginificantly.

**Categorial values**
For the columns with categorial data, we opted to impute these missing entries using the **mode** of each respective column. The mode is the most frequently occuring value in a data set. This approach is particularly suitable for categorial data since it ensures that the imputed values are actual categories that appear in the data. Also this allows us to preserve the distribution of categories within the dataset.